In [1]:
import numpy as np
import pandas as pd
from constants import SHARED_RANDOM_STATE
from db_helper_functions import (
    get_stock_news_with_finbert_scores_from_db,
    get_stock_news_with_finbert_tone_scores_from_db,
    get_stock_news_with_finbert_whole_article_scores_from_db,
    get_stock_news_with_finbert_tone_whole_article_scores_from_db,
    get_stock_news_with_bertopic_sentiment_scores_from_db,
)
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from supervised_learning_data_preprocessing_functions import (
    gen_df_for_supervised_learning,
)
from grid_model_search_functions import iterative_grid_cv_model_testing

ticker = "AAPL"

/var/folders/0r/w8vqcgg56n9113ksf0mt1n940000gn/T/ipykernel_78750/868030337.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
finbert_summary_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_scores_from_db,
)
finbert_tone_summary_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_tone_scores_from_db,
)
finbert_whole_article_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_whole_article_scores_from_db,
)
finbert_tone_article_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_finbert_tone_whole_article_scores_from_db,
)
bert_topic_article_sentiment_df = gen_df_for_supervised_learning(
    ticker=ticker,
    sentiment_df_retrieval_function=get_stock_news_with_bertopic_sentiment_scores_from_db,
)

/Users/raulmartinez/Desktop/MADS/Milestone_2/w24-milestone2-team18-leevw-steveso-raulmart/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/raulmartinez/Desktop/MADS/Milestone_2/w24-milestone2-team18-leevw-steveso-raulmart/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/raulmartinez/Desktop/MADS/Milestone_2/w24-milestone2-team18-leevw-steveso-raulmart/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.

In [3]:
dataframes_to_test = [
    (finbert_summary_sentiment_df, "finbert_summary_sentiment_df"),
    (finbert_tone_summary_sentiment_df, "finbert_tone_summary_sentiment_df"),
    (finbert_whole_article_sentiment_df, "finbert_whole_article_sentiment_df"),
    (finbert_tone_article_sentiment_df, "finbert_tone_whole_article_sentiment_df"),
    (bert_topic_article_sentiment_df, "bert_topic_article_sentiment_df"),
]
cv_train_size = [10, 20, 60, 252, 504]
cv_test_size = [5, 10, 60, 120]
lag_time = [0, 5, 30]
features_to_use = [
    "open",
    "prev_high",
    "prev_low",
    "prev_close",
    # "prev_volume",
    # "dividends",
    # "stock_splits",
    "positive",
    "negative",
    "neutral",
    # "day_of_month",
    # "day_of_week",
    # "quarter",
    # "month",
    # "year",
]
feature_to_predict = ["close"]
scoring_method = ["neg_mean_absolute_error"]

param_grid = {
    "data_frame": dataframes_to_test,
    "cv_train_size": cv_train_size,
    "cv_test_size": cv_test_size,
    "lag_time": lag_time,
    "feature_to_predict": feature_to_predict,
    "scoring_method": scoring_method,
}
data_settings_grid_list = list(ParameterGrid(param_grid))

In [4]:
knn_results = iterative_grid_cv_model_testing(
    model=KNeighborsRegressor(),
    model_parameters={"n_neighbors": [2, 3, 4, 5]},
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
print(len(knn_results))
knn_results.sort_values("mean_test_score", ascending=False).head()

2500


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,param_model__p,params,split0_test_score,split1_test_score,split2_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
3,0.001081,0.000107,0.000655,0.000073,5,1,"{'model__n_neighbors': 5, 'model__p': 1}",-2.362925,-2.232975,-2.434821,...,-3.276525,-2.519181,0.548800,1,finbert_tone_article_sentiment_df,252,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
2,0.001243,0.000107,0.000692,0.000032,4,1,"{'model__n_neighbors': 4, 'model__p': 1}",-1.972594,-1.988757,-3.001805,...,-2.662168,-2.519777,0.579761,1,finbert_tone_article_sentiment_df,504,15,5,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
3,0.001213,0.000085,0.000691,0.000051,5,1,"{'model__n_neighbors': 5, 'model__p': 1}",-2.294795,-2.013992,-2.834135,...,-2.601449,-2.528514,0.542876,2,finbert_tone_article_sentiment_df,504,15,5,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
2,0.001132,0.000107,0.000659,0.000055,4,1,"{'model__n_neighbors': 4, 'model__p': 1}",-2.572367,-1.650667,-2.809730,...,-3.627752,-2.543435,0.620915,1,finbert_tone_article_sentiment_df,252,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
2,0.001508,0.000457,0.000754,0.000076,4,1,"{'model__n_neighbors': 4, 'model__p': 1}",-1.684749,-2.765640,-2.739420,...,-2.924476,-2.551930,0.658498,1,finbert_tone_article_sentiment_df,504,10,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
knn_results.to_csv("./knn_regression_results.csv")

In [ ]:
gbr_results = iterative_grid_cv_model_testing(
    model=GradientBoostingRegressor(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "learning_rate": [0.01, 0.1, 1],
        "n_estimators": [50, 100, 300],
        "max_depth": [None, 2, 5],
        "max_leaf_nodes": [None, 5, 10],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
print(len(gbr_results))
gbr_results.sort_values("mean_test_score", ascending=False).head()

50625


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__learning_rate,param_model__max_depth,param_model__max_leaf_nodes,param_model__n_estimators,param_model__random_state,params,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
33,0.061975,0.003848,0.000753,0.000025,0.1,None,10,50,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,-2.365498,-2.215701,0.531494,1,finbert_tone_article_sentiment_df,504,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
34,0.148579,0.008524,0.000790,0.000022,0.1,None,10,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,-2.317530,-2.240229,0.534359,2,finbert_tone_article_sentiment_df,504,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
37,0.075027,0.006582,0.000989,0.000482,0.1,2,None,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,-2.591704,-2.254601,0.579071,1,finbert_tone_article_sentiment_df,504,15,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
43,0.083531,0.011218,0.001546,0.002181,0.1,2,10,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,-2.591704,-2.254601,0.579071,1,finbert_tone_article_sentiment_df,504,15,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
40,0.075951,0.005878,0.000835,0.000194,0.1,2,5,100,1337,"{'model__learning_rate': 0.1, 'model__max_dept...",...,-2.591704,-2.254601,0.579071,1,finbert_tone_article_sentiment_df,504,15,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
gbr_results.to_csv("./model_results/gb_regression_results.csv")

In [ ]:
rf_results = iterative_grid_cv_model_testing(
    model=RandomForestRegressor(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "n_estimators": [10, 20, 100],
        "max_depth": [None, 2, 5],
        "max_leaf_nodes": [None, 5, 10],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
rf_results.to_csv("./model_results/rf_regression_results.csv")

In [ ]:
rf_results.sort_values("mean_test_score", ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__max_leaf_nodes,param_model__n_estimators,param_model__random_state,params,split0_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
20,0.124194,0.013436,0.002304,0.000757,5,None,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",-2.462029,...,-2.395014,-2.272885,0.479005,1,finbert_tone_article_sentiment_df,504,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
19,0.023704,0.002990,0.001258,0.000867,5,None,20,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",-2.414791,...,-2.397821,-2.274320,0.449728,2,finbert_tone_article_sentiment_df,504,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
19,0.024353,0.004372,0.001054,0.000198,5,None,20,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",-2.411198,...,-2.622234,-2.274383,0.544866,1,finbert_tone_article_sentiment_df,504,15,20,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
1,0.046219,0.018826,0.002709,0.003723,None,None,20,1337,"{'model__max_depth': None, 'model__max_leaf_no...",-2.440908,...,-2.568075,-2.293425,0.421620,3,finbert_tone_article_sentiment_df,504,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
18,0.013574,0.003035,0.001468,0.001002,5,None,10,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",-2.343246,...,-2.455403,-2.294955,0.504758,4,finbert_tone_article_sentiment_df,504,15,60,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
lin_reg_results = iterative_grid_cv_model_testing(
    model=LinearRegression(),
    model_parameters={},
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
print(len(lin_reg_results))
lin_reg_results.sort_values("mean_test_score", ascending=False).head()

625


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
0,0.001334,0.000091,0.000499,0.000019,{},-2.032348,-1.423444,-1.371791,-1.213113,-1.307788,...,-2.460166,-1.866182,0.479337,1,finbert_tone_article_sentiment_df,504,60,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001324,0.000085,0.000491,0.000022,{},-2.029319,-1.403279,-1.377488,-1.241795,-1.311704,...,-2.478244,-1.866664,0.475687,1,finbert_summary_sentiment_df,504,60,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001244,0.000112,0.000506,0.000051,{},-2.035644,-1.406769,-1.365460,-1.208647,-1.312731,...,-2.451350,-1.866819,0.480117,1,finbert_tone_article_sentiment_df,504,60,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001247,0.000121,0.000488,0.000012,{},-2.041573,-1.432794,-1.371827,-1.206308,-1.305426,...,-2.468679,-1.867009,0.479059,1,finbert_tone_summary_sentiment_df,504,60,15,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001260,0.000110,0.000479,0.000012,{},-2.048549,-1.403492,-1.371400,-1.236804,-1.323817,...,-2.458205,-1.870727,0.478331,1,finbert_tone_article_sentiment_df,504,60,20,neg_mean_absolute_error,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
lin_reg_results.to_csv("./model_results/linear_regression_results.csv")

In [ ]:
for df, name in dataframes_to_test:
    df["closed_higher"] = np.where(df.close > df.open, 1, 0)


feature_to_predict = ["closed_higher"]
scoring_method = ["accuracy"]

param_grid = {
    "data_frame": dataframes_to_test,
    "cv_train_size": cv_train_size,
    "cv_test_size": cv_test_size,
    "lag_time": lag_time,
    "feature_to_predict": feature_to_predict,
    "scoring_method": scoring_method,
}
data_settings_grid_list = list(ParameterGrid(param_grid))

In [ ]:
lr_clf_results = iterative_grid_cv_model_testing(
    model=LogisticRegression(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
lr_clf_results.sort_values("mean_test_score", ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
0,0.001779,0.000107,0.000606,0.000030,1337,{'model__random_state': 1337},0.6,0.6,0.6,0.6,...,0.4,0.66,0.156205,1,finbert_tone_summary_sentiment_df,20,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001768,0.000077,0.000621,0.000027,1337,{'model__random_state': 1337},0.6,0.8,0.6,0.6,...,0.4,0.66,0.128062,1,finbert_tone_summary_sentiment_df,10,5,15,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001763,0.000124,0.000607,0.000029,1337,{'model__random_state': 1337},0.6,0.8,0.6,0.6,...,0.4,0.64,0.149666,1,finbert_summary_sentiment_df,20,5,15,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001975,0.000158,0.000688,0.000071,1337,{'model__random_state': 1337},0.6,0.4,0.6,0.8,...,0.4,0.62,0.188680,1,finbert_whole_article_sentiment_df,60,5,20,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.001756,0.000159,0.000603,0.000031,1337,{'model__random_state': 1337},0.4,0.6,0.6,0.6,...,0.4,0.62,0.166132,1,finbert_tone_summary_sentiment_df,15,5,15,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
lr_clf_results.to_csv("./model_results/logistic_regression_classifier_results.csv")

In [ ]:
gbr_clf_results = iterative_grid_cv_model_testing(
    model=GradientBoostingClassifier(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "learning_rate": [0.1],
        "n_estimators": [100, 300],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
gbr_clf_results.sort_values("mean_test_score", ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__learning_rate,param_model__n_estimators,param_model__random_state,params,split0_test_score,split1_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
1,0.075393,0.010646,0.000937,0.000036,0.1,300,1337,"{'model__learning_rate': 0.1, 'model__n_estima...",0.6,0.6,...,0.8,0.66,0.156205,1,finbert_tone_summary_sentiment_df,20,5,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
1,0.101812,0.007924,0.001040,0.000348,0.1,300,1337,"{'model__learning_rate': 0.1, 'model__n_estima...",0.8,0.6,...,0.4,0.66,0.200998,1,finbert_tone_summary_sentiment_df,60,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.031244,0.001907,0.000949,0.000249,0.1,100,1337,"{'model__learning_rate': 0.1, 'model__n_estima...",0.8,0.6,...,0.4,0.64,0.233238,2,finbert_tone_summary_sentiment_df,60,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.024085,0.001684,0.000830,0.000032,0.1,100,1337,"{'model__learning_rate': 0.1, 'model__n_estima...",1.0,0.6,...,0.8,0.64,0.174356,1,finbert_summary_sentiment_df,20,5,60,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
0,0.025770,0.001578,0.000874,0.000022,0.1,100,1337,"{'model__learning_rate': 0.1, 'model__n_estima...",0.6,0.6,...,0.6,0.64,0.149666,2,finbert_tone_summary_sentiment_df,20,5,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
gbr_clf_results.to_csv("./model_results/gb_classifier_results.csv")

In [ ]:
rf_clf_results = iterative_grid_cv_model_testing(
    model=RandomForestClassifier(),
    model_parameters={
        "random_state": [SHARED_RANDOM_STATE],
        "n_estimators": [10, 20, 100],
        "max_depth": [None, 2, 5],
        "max_leaf_nodes": [None, 5, 10],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

In [ ]:
rf_clf_results.sort_values("mean_test_score", ascending=False).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__max_leaf_nodes,param_model__n_estimators,param_model__random_state,params,split0_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
2,0.077501,0.015053,0.002465,0.000941,None,None,100,1337,"{'model__max_depth': None, 'model__max_leaf_no...",0.4,...,0.6,0.72,0.160000,1,finbert_tone_article_sentiment_df,252,5,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
23,0.052557,0.006565,0.002769,0.001453,5,5,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.6,...,1.0,0.68,0.222711,1,finbert_tone_summary_sentiment_df,15,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
5,0.049194,0.005185,0.002163,0.000313,None,5,100,1337,"{'model__max_depth': None, 'model__max_leaf_no...",0.6,...,1.0,0.68,0.222711,1,finbert_tone_summary_sentiment_df,15,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
22,0.012205,0.002750,0.001193,0.000141,5,5,20,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.6,...,0.6,0.68,0.097980,1,finbert_tone_summary_sentiment_df,60,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
20,0.056085,0.011179,0.002227,0.000205,5,None,100,1337,"{'model__max_depth': 5, 'model__max_leaf_nodes...",0.6,...,0.6,0.68,0.203961,1,finbert_tone_summary_sentiment_df,60,5,10,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."


In [ ]:
rf_clf_results.to_csv("./model_results/rf_classifier_results.csv")

In [ ]:
knn_clf_results = iterative_grid_cv_model_testing(
    model=KNeighborsClassifier(),
    model_parameters={
        "n_neighbors": [2, 3, 4, 5],
    },
    data_settings_grid_list=data_settings_grid_list,
    features_to_use=features_to_use,
)

NameError: name 'iterative_grid_cv_model_testing' is not defined

In [ ]:
knn_clf_results[knn_clf_results.cv_test_size].sort_values(
    "mean_test_score", ascending=False
).head()

NameError: name 'knn_clf_results' is not defined

In [ ]:
knn_clf_results = pd.read_csv("./model_results/knn_classifier_results.csv")

In [ ]:
knn_clf_results[knn_clf_results.cv_test_size > 15].sort_values(
    "mean_test_score", ascending=False
).head()

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,...,split9_test_score,mean_test_score,std_test_score,rank_test_score,data_frame,cv_train_size,cv_test_size,lag_time,scoring_method,features_to_use
1816,0,0.001328,0.000094,0.001282,0.000074,2,{'model__n_neighbors': 2},0.55,0.55,0.75,...,0.65,0.610,0.091652,1,finbert_summary_sentiment_df,20,20,60,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
2041,1,0.001287,0.000072,0.001291,0.000119,3,{'model__n_neighbors': 3},0.55,0.55,0.65,...,0.50,0.595,0.082006,1,finbert_whole_article_sentiment_df,252,20,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
2155,3,0.001317,0.000121,0.001269,0.000082,5,{'model__n_neighbors': 5},0.60,0.65,0.70,...,0.45,0.590,0.139284,1,finbert_whole_article_sentiment_df,504,20,20,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
2053,1,0.001233,0.000081,0.001237,0.000047,3,{'model__n_neighbors': 3},0.65,0.55,0.70,...,0.45,0.590,0.109087,1,finbert_whole_article_sentiment_df,252,20,20,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
2141,1,0.001369,0.000127,0.001309,0.000062,3,{'model__n_neighbors': 3},0.55,0.55,0.75,...,0.45,0.585,0.122577,1,finbert_whole_article_sentiment_df,504,20,5,accuracy,"open,prev_high,prev_low,prev_close,positive,ne..."
